In [11]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
import asyncio
from email.mime.text import MIMEText
import smtplib

load_dotenv(override=True)

True

In [8]:

def send_test_email():
    body = "This is automated email"
    from_email = os.getenv("GMAIL_USER", "your_email@gmail.com")  # Replace with your Gmail address or set as env var
    to_email = os.getenv("GMAIL_TO", "recipient@example.com")     # Replace with recipient or set as env var
    gmail_app_password = os.getenv("GMAIL_APP_PASSWORD", "your_app_password")  # Use your Gmail app password or set as env var
    subject = "Sales email"

    # Create the email
    msg = MIMEText(body, 'html')
    msg['Subject'] = subject
    msg['From'] = from_email
    msg['To'] = to_email

    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()  # Secure the connection
            server.login(from_email, gmail_app_password)
            server.send_message(msg)
        status = {"status": "success"}
        print(status)
        # return status
        
    except Exception as e:
        # return {"status": "failure", "message": str(e)}
        print(e)

send_test_email()

{'status': 'success'}


In [9]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [12]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [13]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Simplify Your SOC 2 Compliance with AI-Powered Solutions

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m reaching out to introduce you to ComplAI, a cutting-edge SaaS platform designed to streamline SOC 2 compliance and simplify audit preparations.

In today’s rapidly evolving regulatory landscape, ensuring compliance can be a daunting task. Our AI-driven tool not only helps you maintain compliance but also reduces the time and resources necessary for audits. 

Here are a few key benefits of ComplAI:

- **Automated Compliance Monitoring**: Stay on top of your compliance requirements with real-time tracking and automated alerts.
- **User-Friendly Interface**: Easily navigate the complexities of SOC 2 with our intuitive dashboard.
- **Robust Documentation Support**: Generate necessary documentation at the click of a button, ensuring you're always audit-ready.

I would love to schedule a brief call to discuss how ComplAI can support 

In [14]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

Subject: Simplify Your SOC 2 Compliance Process

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I represent ComplAI, a leading provider of AI-powered SaaS solutions designed to streamline SOC 2 compliance and audit preparation.

In today’s fast-paced regulatory environment, maintaining compliance can be both time-consuming and challenging. Our platform automates various compliance processes, enabling your team to focus on what they do best while ensuring that you meet all necessary requirements efficiently.

Here are a few ways ComplAI can support your compliance journey:

1. **Automated Documentation**: Generate and maintain essential compliance documents without the manual hassle.
2. **Real-time Monitoring**: Stay updated on compliance status and receive alerts on any discrepancies.
3. **Audit Readiness**: Easily prepare for audits with comprehensive reporting and insights at your fingertips.

I would love the opportunity to discuss how Comp

In [15]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [16]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")

Best sales email:
Subject: 🚀 Ready to Turn Your SOC2 Compliance from “Oops!” to “Oops… Who?”  

Hey [Recipient's Name],  

I hope this email finds you swimming in compliance documents and not drowning in them! If your SOC2 compliance feels more like a circus act—juggling audits, tight deadlines, and unexpected regulations—I’ve got the perfect act for you. 🎪  

I’m reaching out from ComplAI, where we’ve harnessed the power of AI to tackle those SOC2 compliance headaches faster than you can say “regulatory requirements.” Imagine preparing for an audit without the usual panic attacks; it’s like finding out that your favorite snack has zero calories (okay, maybe not *that* good, but close!).  

Here’s the deal: our tool automagically ensures your compliance is tight, keeping the auditors happy and your team focused on what really matters—like finally implementing that coffee machine you’ve been dreaming about. ☕   

How about a quick chat to see if we can swap your compliance woes for some

# Part 2: use of tools

sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)

In [17]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects via Gmail SMTP """
    
    # Set up email sender, recipient, and content
    from_email = os.getenv("GMAIL_USER", "your_email@gmail.com")  # Replace with your Gmail address or set as env var
    to_email = os.getenv("GMAIL_TO", "recipient@example.com")     # Replace with recipient or set as env var
    gmail_app_password = os.getenv("GMAIL_APP_PASSWORD", "your_app_password")  # Use your Gmail app password or set as env var
    subject = "Sales email"
    
    # Create the email
    msg = MIMEText(body, 'html')
    msg['Subject'] = subject
    msg['From'] = from_email
    msg['To'] = to_email
    
    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()  # Secure the connection
            server.login(from_email, gmail_app_password)
            server.send_message(msg)
        return {"status": "success"}
    except Exception as e:
        return {"status": "failure", "message": str(e)}


In [18]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10d2284c0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10d22a560>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required'

In [20]:
instructions ="You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales_agent tools once before choosing the best one. \
Use vietnamese language. \
You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user"


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

## Handoffs represent a way an agent can delegate to an agent, passing control to it

In [21]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [22]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given body to all sales prospects via Gmail SMTP """
    
    # Set up email sender, recipient, and content
    from_email = os.getenv("GMAIL_USER", "your_email@gmail.com")  # Replace with your Gmail address or set as env var
    to_email = os.getenv("GMAIL_TO", "recipient@example.com")     # Replace with recipient or set as env var
    gmail_app_password = os.getenv("GMAIL_APP_PASSWORD", "your_app_password")  # Use your Gmail app password or set as env var
    # subject = "Sales email"
    
    # Create the email
    msg = MIMEText(html_body, 'html')
    msg['Subject'] = subject
    msg['From'] = from_email
    msg['To'] = to_email
    
    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()  # Secure the connection
            server.login(from_email, gmail_app_password)
            server.send_message(msg)
        return {"status": "success"}
    except Exception as e:
        return {"status": "failure", "message": str(e)}

In [23]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [27]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10d2284c0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10d22a560>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': 

In [28]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)